In [1]:
from pathlib import Path
Path('my_data.db').touch()
import sqlite3
con = sqlite3.connect('my_data.db')
cur = con.cursor()
sqlite3.sqlite_version

'3.37.2'

In [2]:
cur.execute("DROP TABLE Products")
cur.execute("DROP TABLE Orders")
cur.execute("DROP TABLE Order_Items")
cur.execute("DROP TABLE Reviews")
cur.execute("DROP TABLE Categories")
cur.execute("DROP TABLE Users")

In [3]:
import pandas as pd

Products = pd.read_csv("product_data.csv")
Orders = pd.read_csv("order_data.csv")
Order_Items = pd.read_csv("order_items_data.csv")
Reviews = pd.read_csv("review_data.csv")
Categories = pd.read_csv("category_data.csv")
Users = pd.read_csv("user_data.csv")

-- Modifications for testing queries

In [4]:
Categories = Categories[Categories['category_id'].isin(Products['category_id'].values)]

In [5]:
Reviews = Reviews[Reviews['product_id'] != 5]

In [6]:
new_orders = {
    'order_id': [31,32,33,34,35,36,37,38],
    'user_id': [4,4,4,4,4,4,4,4],
    'order_date': ['2021-04-21','2021-04-19', '2022-01-02', '2022-02-03', '2022-03-04', '2022-04-05', '2022-05-06', '2022-06-07'],
    'total_amount': [500,200,0,0,0,0,0,0]
}

new_order_items = {
    'order_item_id': [31,32,33,34,35,36,37,38],
    'order_id': [31,32,33,34,35,36,37,38],
    'product_id': [2,4,6,8,10,12,14,16],
    'quantity': [0,0,0,0,0,0,0,0],
    'unit_price': [0,0,0,0,0,0,0,0],
}

Orders = pd.concat([Orders, pd.DataFrame(new_orders)], ignore_index=True).reset_index()
Order_Items = pd.concat([Order_Items, pd.DataFrame(new_order_items)], ignore_index=True).reset_index()

In [7]:
new_review = {"review_id": 31, "user_id": 6, "product_id": 6, "rating": 3, "review_text": "Meh product", "review_date": '2022-05-04'}
Reviews = Reviews.append(new_review, ignore_index = True)

<ipython-input-7-d1cb63c9c4e9>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Reviews = Reviews.append(new_review, ignore_index = True)


In [8]:
Products.to_sql('Products', con,  index=False)
Orders.to_sql('Orders', con,  index=False)
Order_Items.to_sql('Order_Items', con,  index=False)
Reviews.to_sql('Reviews', con,  index=False)
Categories.to_sql('Categories', con,  index=False)
Users.to_sql('Users', con,  index=False)

30

-- Problem 1: Retrieve all products in the Sports category
-- Write an SQL query to retrieve all products in a specific category.

In [9]:
pd.read_sql_query("""SELECT p.product_id, p.product_name, p.description, p.price
FROM Products p LEFT JOIN Categories c ON p.category_id = c.category_id
WHERE c.category_name = 'Sports & Outdoors';""", con)

,product_id,product_name,description,price
0,15,Mountain Bike,Conquer the trails with this high-performance ...,1000.0
1,16,Tennis Racket,Take your tennis game to the next level with t...,54.0


-- Problem 2: Retrieve the total number of orders for each user
-- Write an SQL query to retrieve the total number of orders for each user.
-- The result should include the user ID, username, and the total number of orders.

In [10]:
pd.read_sql_query("""SELECT u.user_id, u.username, COUNT(o.order_id) AS order_count
FROM Users u JOIN Orders o ON u.user_id = o.user_id
GROUP BY u.user_id;""", con)

# user_id: 4 -> order_count: 1+8=9 (rest 1s)

,user_id,username,order_count
0,1,johndoe,1
1,2,janesmith,1
2,3,maryjones,1
3,4,robertbrown,9
4,5,sarahwilson,1
5,6,michaellee,1
6,7,lisawilliams,1
7,8,chrisharris,1
8,9,emilythompson,1
9,10,davidmartinez,1


-- Problem 3: Retrieve the average rating for each product
-- Write an SQL query to retrieve the average rating for each product.
-- The result should include the product ID, product name, and the average rating.

In [11]:
pd.read_sql_query("""SELECT p.product_id, p.product_name, AVG(r.rating) AS avg_rating
FROM Products p JOIN Reviews r ON p.product_id = r.product_id
GROUP BY p.product_id;""", con)

# product_id: 6 -> avg_rating: 3.5 ([4+3]/2)

,product_id,product_name,avg_rating
0,1,Smartphone X,5.0
1,2,Wireless Headphones,4.0
2,3,Laptop Pro,3.0
3,4,Smart TV,5.0
4,6,Designer Dress,3.5
5,7,Coffee Maker,5.0
6,8,Toaster Oven,3.0
7,9,Action Camera,4.0
8,10,Board Game Collection,1.0
9,11,Yoga Mat,5.0


-- Problem 4: Retrieve the top 5 users with the highest total amount spent on orders
-- Write an SQL query to retrieve the top 5 users with the highest total amount spent on orders.
-- The result should include the user ID, username, and the total amount spent.

In [12]:
pd.read_sql_query("""SELECT u.user_id, u.username, SUM(o.total_amount) AS order_total_amount
FROM Users u JOIN Orders o ON u.user_id = o.user_id
GROUP BY u.user_id
ORDER BY order_total_amount DESC
LIMIT 5;""", con)

# user_id: 4 -> order_total_amount = 500+200+155+0*6=855.0

,user_id,username,order_total_amount
0,4,robertbrown,855.0
1,12,jasonrodriguez,160.0
2,24,jamesrogers,150.0
3,8,chrisharris,150.0
4,29,elliecollins,145.0


-- Problem 5: Retrieve the products with the highest average rating
-- Write an SQL query to retrieve the products with the highest average rating.
-- The result should include the product ID, product name, and the average rating.
-- Hint: You may need to use subqueries or common table expressions (CTEs) to solve this problem.

In [13]:
pd.read_sql_query("""WITH AverageRatings AS (
    SELECT p.product_id, p.product_name, AVG(r.rating) AS avg_rating
    FROM Products p JOIN Reviews r ON p.product_id = r.product_id
    GROUP BY p.product_id
)
SELECT * FROM AverageRatings WHERE avg_rating = (SELECT MAX(avg_rating) FROM AverageRatings);""", con)

,product_id,product_name,avg_rating
0,1,Smartphone X,5.0
1,4,Smart TV,5.0
2,7,Coffee Maker,5.0
3,11,Yoga Mat,5.0
4,15,Mountain Bike,5.0


-- Problem 6: Retrieve the users who have made at least one order in each category
-- Write an SQL query to retrieve the users who have made at least one order in each category.
-- The result should include the user ID and username.
-- Hint: You may need to use subqueries or joins to solve this problem.

In [14]:
pd.read_sql_query("""SELECT u.user_id, u.username
FROM Users u JOIN Orders o ON u.user_id = o.user_id JOIN Order_Items oi ON o.order_id = oi.order_id JOIN Products p ON oi.product_id = p.product_id
GROUP BY u.user_id HAVING COUNT(DISTINCT p.category_id) = (SELECT COUNT(*) FROM Categories);""", con)

# user_id: 4 (only user with one order in each category)

,user_id,username
0,4,robertbrown


-- Problem 7: Retrieve the products that have not received any reviews
-- Write an SQL query to retrieve the products that have not received any reviews.
-- The result should include the product ID and product name.
-- Hint: You may need to use subqueries or left joins to solve this problem.

In [15]:
pd.read_sql_query("""SELECT p.product_id, p.product_name
FROM Products p LEFT JOIN Reviews r ON p.product_id = r.product_id
WHERE r.review_id IS NULL;""", con)

# product_id: 5 (only one with no reviews)

,product_id,product_name
0,5,Running Shoes


-- Problem 8: Retrieve the users who have made consecutive orders on consecutive days
-- Write an SQL query to retrieve the users who have made consecutive orders on consecutive days.
-- The result should include the user ID and username.
-- Hint: You may need to use subqueries or window functions to solve this problem.

In [16]:
pd.read_sql_query("""WITH ConsecutiveOrders AS (
    SELECT o.user_id, o.order_date,
    LAG(o.order_date) OVER (PARTITION BY o.user_id ORDER BY o.order_date) AS prev_order_date
    FROM Orders o
)
SELECT DISTINCT c.user_id, u.username
FROM ConsecutiveOrders c JOIN Users u ON c.user_id = u.user_id
WHERE c.order_date = date(c.prev_order_date, '+1 day');""", con)

# user_id: 4 (only user with consecutive orders)

,user_id,username
0,4,robertbrown


-- Problem 9: Retrieve the top 3 categories with the highest total sales amount
-- Write an SQL query to retrieve the top 3 categories with the highest total sales amount.
-- The result should include the category ID, category name, and the total sales amount.
-- Hint: You may need to use subqueries, joins, and aggregate functions to solve this problem.

In [17]:
pd.read_sql_query("""SELECT c.category_id, c.category_name, SUM(oi.quantity * oi.unit_price) AS total_sales
FROM Categories c JOIN Products p ON c.category_id = p.category_id JOIN Order_Items oi ON p.product_id = oi.product_id
GROUP BY c.category_id
ORDER BY total_sales DESC
LIMIT 3;""", con)

,category_id,category_name,total_sales
0,8,Sports & Outdoors,155.0
1,4,Home & Kitchen,145.0
2,1,Electronics,125.0


-- Problem 10: Retrieve the users who have placed orders for all products in the Toys & Games
-- Write an SQL query to retrieve the users who have placed orders for all products in the Toys & Games
-- The result should include the user ID and username.
-- Hint: You may need to use subqueries, joins, and aggregate functions to solve this problem.

In [18]:
pd.read_sql_query("""SELECT u.user_id, u.username
FROM Users u JOIN Orders o ON u.user_id = o.user_id JOIN Order_Items oi ON o.order_id = oi.order_id JOIN Products p ON oi.product_id = p.product_id JOIN Categories c ON p.category_id = c.category_id
WHERE c.category_name = 'Toys & Games'
GROUP BY u.user_id
HAVING COUNT(DISTINCT p.product_id) = (SELECT COUNT(*) FROM Products WHERE category_id = (SELECT category_id FROM Categories WHERE category_name = 'Toys & Games'));""", con)

,user_id,username
0,5,sarahwilson


-- Problem 11: Retrieve the products that have the highest price within each category
-- Write an SQL query to retrieve the products that have the highest price within each category.
-- The result should include the product ID, product name, category ID, and price.
-- Hint: You may need to use subqueries, joins, and window functions to solve this problem.

In [19]:
pd.read_sql_query("""SELECT p.product_id, p.product_name, p.category_id, p.price
FROM Products p
WHERE (p.category_id, p.price) IN (SELECT category_id, MAX(price) FROM Products GROUP BY category_id);""", con)

,product_id,product_name,category_id,price
0,1,Smartphone X,1,500.0
1,3,Laptop Pro,2,1200.0
2,6,Designer Dress,3,300.0
3,7,Coffee Maker,4,80.0
4,9,Action Camera,5,200.0
5,12,Skincare Set,6,150.0
6,14,Weighted Blanket,7,100.0
7,15,Mountain Bike,8,1000.0


-- Problem 12: Retrieve the users who have placed orders on consecutive days for at least 3 days
-- Write an SQL query to retrieve the users who have placed orders on consecutive days for at least 3 days.
-- The result should include the user ID and username.
-- Hint: You may need to use subqueries, joins, and window functions to solve this problem.

In [20]:
pd.read_sql_query("""WITH ConsecutiveOrders AS (
    SELECT o.user_id, o.order_date,
    LAG(o.order_date) OVER (PARTITION BY o.user_id ORDER BY o.order_date) AS prev_order_date,
    LEAD(o.order_date) OVER (PARTITION BY o.user_id ORDER BY o.order_date) AS next_order_date
    FROM Orders o
)
SELECT DISTINCT c.user_id, u.username
FROM ConsecutiveOrders c JOIN Users u ON c.user_id = u.user_id
WHERE c.order_date = date(c.prev_order_date, '+1 day') AND c.order_date = date(c.next_order_date, '-1 day');""", con)

# user_id: 4 (only user with 3 consecutive orders)

,user_id,username
0,4,robertbrown
